In [ ]:
import pandas as pd
import numpy as np
from io import StringIO
from io import BytesIO

!pip install azure-storage-blob
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

In [ ]:
#defining azure connection string, container name & blob name
CONNECTION_STRING_AZURE_STORAGE='DefaultEndpointsProtocol=https;AccountName=affordablehousing;AccountKey=J+3/tpTlA4eRSpu7TOQ+zSx7jErO15HRSGZa/e2iLxVDiFzjMP9njTT71S5W284vk4EjNA4i36+y+AStS5uxew==;EndpointSuffix=core.windows.net'
CONTAINER_AZURE='affordablehousing'

#creating azure functions to upload to the blob
def azure_upload_blob(connect_str, container_name, blob_name, data):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=CONTAINER_AZURE, blob=blob_name)
    blob_client.upload_blob(data, overwrite=True)
    print(f"Uploaded to Azure Blob: {blob_name}")
    
#creating azure functions 
def azure_download_blob(connect_str, container_name, blob_name):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=CONTAINER_AZURE, blob=blob_name)
    download_stream = blob_client.download_blob()
    return download_stream.readall()

In [ ]:
#downloading csv from azure storage
download_data=azure_download_blob(CONNECTION_STRING_AZURE_STORAGE, CONTAINER_AZURE, 'raw_affordablehousingdata.csv')

In [ ]:
#converting csv download into df
download_data=pd.read_csv(BytesIO(download_data))
download_data

In [ ]:
download_data.info()

In [ ]:
#dropping census_tract since all null values
download_data=download_data.drop(columns=['census_tract'])
print(download_data.columns)

In [ ]:
#removing any duplicate building_id values
download_data=download_data.drop_duplicates(subset=['building_id'])
download_data.info()

In [ ]:
#dropping null values in building_id 
download_data=download_data.dropna(subset=['building_id'])
download_data.info()

In [ ]:
#allows to write over copied dataframe
pd.options.mode.copy_on_write = True

In [ ]:
#converting building_id from float to int 
download_data['building_id']=download_data['building_id'].astype(int)
download_data

In [ ]:
#dropping rows with null values in building_completion_date since we are only interested in completed buildings for analysis
download_data=download_data.dropna(subset=['building_completion_date'])
download_data.info()

In [ ]:
#dropping rows with null values in project_completion_date and project_start_date since we are only interested in completed projects for analysis
download_data=download_data.dropna(subset=['project_completion_date','project_start_date'])
download_data.info()

In [ ]:
#dropping null values for postcode
download_data=download_data.dropna(subset=['postcode'])
download_data.info()

In [ ]:
#creating new column for total_low_income_units
download_data['total_low_income_units']=download_data['extremely_low_income_units']+download_data['very_low_income_units']+download_data['low_income_units']
download_data.info()

In [ ]:
#creating new column for total_non-low_income_units
download_data['total_non_low_income_units']=download_data['moderate_income_units']+download_data['middle_income_units']
download_data.info()

In [ ]:
#dropping columns not needed for analysis, these columns are dropped because I will only focus on low income vs non-low income per location 
download_data=download_data.drop(columns=['project_start_date','project_completion_date','bbl','bin','community_board','council_district','neighborhood_tabulation_area',
                                          'latitude_internal','longitude_internal','building_completion_date','extended_affordability_status','prevailing_wage_status',
                                            'other_income_units','unknown_br_units'])
download_data.info()

In [ ]:
# convert dataframe to csv
output = StringIO()
download_data.to_csv(output, index=False)
data = output.getvalue()
output.close()
blob_name = "cleaned_affordablehousingdata.csv"

In [ ]:
#blob client creation
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)
blob_client = blob_service_client.get_blob_client(container=CONTAINER_AZURE, blob=blob_name)

#uploading csv data
blob_client.upload_blob(data, overwrite=True)